In [ ]:
################################################
# New style 
################################################
import sys
import os

# Cartopy for pretty maps
# Done first because of compatibilty issues
# with ESMF
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature


workdir_ = '/glade/work/juliob'
if ( workdir_ not in sys.path ):
    sys.path.append(workdir_)
    print( f" a path to {workdir_} added in {__name__} ")

from myPythonTools.Utils import utils as uti
from myPythonTools.Utils import numerical_utils as nuti
from myPythonTools.Utils import AveragingUtils as Av
from myPythonTools.Utils import validation_data as Val
from myPythonTools.Utils import PlotUtil as Pu
from myPythonTools.Plotting import LatLonMaps as LL 
from myPythonTools.CASutils import filter_utils as fu 

from PyRegridding.Utils import GridUtils as GrU
from PyRegridding.Utils import MakePressures as MkP
from PyRegridding.Drivers import RegridField as RgF
from PyRegridding.Regridder import scripGen


# The usual
from datetime import date
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.tri as tri

# Some other useful packages 
import importlib
import copy
import time
import cftime

importlib.reload( uti )
importlib.reload( nuti )
importlib.reload( Pu )
importlib.reload(Av)
importlib.reload(Val)
#importlib.reload(vAB)
importlib.reload(MkP)
importlib.reload(RgF)
importlib.reload(LL)


try:
    RgObs
except NameError:
    RgObs = {}
# OR ???
#RgObs=globals().get("RgObs", {} )          


In [ ]:
fn='/glade/derecho/scratch/juliob/archive/c64_48_gwR2_ne30pg3_FMTHIST_devel_02/atm/hist/c64_48_gwR2_ne30pg3_FMTHIST_devel_02.cam.h1i.1985-07-15-00000.nc'
fn2='/glade/derecho/scratch/juliob/archive/c64_48_gwR2_ne30pg3_FMTHIST_devel_04/atm/hist/c64_48_gwR2_ne30pg3_FMTHIST_devel_04.cam.h1i.1985-07-15-00000.nc'


X=xr.open_dataset( fn )
X2=xr.open_dataset( fn2 )


u=X.U.values
v=X.V.values
zeta=X.VORT4GW.values
u2=X2.U.values


In [ ]:
print( X.lev[91-10].values )


In [ ]:

plt.plot(u[0,20,:])
plt.plot((u2-u)[0,20,:])


In [ ]:
X

In [ ]:
########################
# Option 2 from unstructured SE data
########################
Src='ne30pg3'
Dst ='fv1x1'

DstInfo = GrU.gridInfo(Dst) #,Vgrid=DstVgrid)
lat_a,lon_a = GrU.latlon( scrip= DstInfo['scrip'], Hkey=DstInfo['Hkey'] )
lev_a=X.lev.values
zlev_a=-7.0*np.log( lev_a/1_000. )


RegridObj = GrU.regrid_object_lib(RgOb=RgObs, src=Src, dst=Dst, RegridMethod='BILINEAR' )

u_yx=RgF.Horz(xfld_Src=u , Src=Src, Dst=Dst , RegridObj_In= RegridObj )
v_yx=RgF.Horz(xfld_Src=v , Src=Src, Dst=Dst , RegridObj_In= RegridObj )
zeta_yx = RgF.Horz(xfld_Src=zeta , Src=Src, Dst=Dst , RegridObj_In= RegridObj )

In [ ]:
zeta_yx.shape

In [ ]:
plt.contourf( zeta_yx[0,90,:,:] )

In [ ]:
nt,nz,ny,nx = np.shape( u_yx )
curluv_yx = np.zeros( (nt,nz,ny,nx ) )

for t in np.arange( nt ):
    for z in np.arange( nz) :
        curluv_yx[t,z,:,:] =nuti.Sphere_Curl2( f_x=u_yx[t,z,:,:] , f_y=v_yx[t,z,:,:] , lat=lat_a , lon=lon_a , wrap=True )

In [ ]:
plt.contourf( curluv_yx[0,90,:,:] )

In [ ]:
dolev = 21 #np.linspace(-1,1,num=21)
olev = 0.0002 * np.linspace(-1,1,num=51)
# Create a figure with a 2x2 grid of subplots
fig, ax = plt.subplots(1, 2, figsize=(20, 5))

# Flatten axs to easily access each subplot by index
ax = ax.flatten()

#GGG
k=70
co=ax[0].contourf(  zeta_yx[0,90,:,:] , levels = olev, cmap='bwr' )
cb=plt.colorbar( co )
co=ax[1].contourf(  curluv_yx[0,90,:,:], levels = olev , cmap='bwr' )
cb=plt.colorbar( co )


In [ ]:

y=30
plt.plot( curluv_yx[0,90,y,:])
plt.plot( zeta_yx[0,90,y,:])


In [ ]:
plt.contourf( lon_a, zlev_a, zeta_yx[0,:,35,:],levels=olev, cmap='bwr' )
plt.ylim(0,30)

In [ ]:
ddoo= (curluv_yx - zeta_yx)**2

In [ ]:
plt.plot( np.average( ddoo[0,40,:,:], axis=1 ) )

In [ ]:
X

In [ ]:

plt.plot(X.XPWP_SRC_MOVMTN.values.flatten() )
plt.plot(10*np.abs(X.VORT4GW[0,80,:].values.flatten() ))


In [ ]:

plt.scatter( X.XPWP_SRC_MOVMTN.values.flatten()  , np.abs(X.VORT4GW[0,92,:].values.flatten() ) )


In [ ]:

#plt.scatter( X.XPWP_SRC_MOVMTN.values.flatten()  , X2.XPWP_SRC_MOVMTN.values.flatten() , s=.1  )
plt.scatter( X.UCELL_MOVMTN.values.flatten()  , X.XPWP_SRC_MOVMTN.values.flatten() , s=.5 , alpha=.2 )
plt.scatter( X2.UCELL_MOVMTN.values.flatten()  , X2.XPWP_SRC_MOVMTN.values.flatten() , s=.5 , alpha=.2 )
#plt.xlim(0,.02)
#plt.ylim(0,.02)



In [ ]:

#plt.scatter( X.XPWP_SRC_MOVMTN.values.flatten()  , X2.XPWP_SRC_MOVMTN.values.flatten() , s=.1  )
plt.scatter( X.UCELL_MOVMTN.values[0,:]  , X.TAU_MOVMTN.values[0,10,:] , s=5 , alpha=.2 )
plt.scatter( X2.UCELL_MOVMTN.values[0,:]  , X2.TAU_MOVMTN.values[0,10,:] , s=5 , alpha=.2 )
#plt.xlim(0,.02)
#plt.ylim(0,.02)



In [ ]:

#plt.scatter( X.XPWP_SRC_MOVMTN.values.flatten()  , X2.XPWP_SRC_MOVMTN.values.flatten() , s=.1  )
#plt.scatter( np.average( np.abs(X2.VORT4GW.values[0,83:,:]),axis=0)  , X2.XPWP_SRC_MOVMTN.values[0,:] , s=5 , alpha=.2 )
#plt.plot( 0.0125*1.e4*np.average( np.abs(X2.VORT4GW.values[0,83:,:]),axis=0)  , alpha=.1 )
#plt.plot( X2.XPWP_SRC_MOVMTN.values[0,:] , alpha=.1 )
plt.plot( 0.0125*1.e4*np.average( np.abs(X2.VORT4GW.values[0,83:,:]),axis=0) - X2.XPWP_SRC_MOVMTN.values[0,:]  )

#plt.xlim(0,.02)
#plt.ylim(0,.02)



In [ ]:
plt.hist( X.XPWP_SRC_MOVMTN.values.flatten(), bins=50 )

In [ ]:

plt.hist( np.abs(X.UCELL_MOVMTN[0,:].values.flatten() ), bins=np.linspace(0,51) ,alpha=.2)
plt.hist( np.abs(X2.UCELL_MOVMTN[0,:].values.flatten() ), bins=np.linspace(0,51), alpha=.2 )


In [ ]:
X2